# Preparation

In [1]:
import sys
import os
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [2]:
import numpy as np
import spacy
from utils import dataset_utils
from utils import metrics

In [3]:
# %%time
# !python -m spacy download ru_core_news_lg

# Data

In [4]:
requests_file = "../data/request_db.txt"
ads_file = "../data/ads_db.txt"
matching_file = "../data/matching_db.txt"

In [5]:
with open(ads_file, encoding="utf-8") as f:
    ads_raw = f.readlines()

In [6]:
with open(requests_file, encoding="utf-8") as f:
    requests_raw = f.readlines()

# Preprocessing

In [7]:
nlp = spacy.load("ru_core_news_lg")

In [8]:
def preprocess(text):
    text = text.replace("\\n", "\n").replace("\n", " ")
    text = text.strip()
    text = text.lower()  # many words has vectors only in lowercase
    return text

In [9]:
ads = [nlp(preprocess(text)) for text in ads_raw]

In [10]:
for ad in ads:
    if ad.vector.all() == 0.:
        print(ad) 

In [11]:
requests = [nlp(preprocess(text)) for text in requests_raw]

In [12]:
for request in requests:
    if not request.has_vector:
        print(request) 

снуд вязанный
дкарандши
беговел
экомешочки
шифонер
электросамокат
самосбор
онтоновка
безсахара
уходовая кометика
вейп
фотостудия
автолюлька
автокресло
чермет


# Predict

In [13]:
def calc_similarity(request, ads):
    return [request.similarity(ad) if ad.has_vector else 0 for ad in ads] if request.has_vector else [0]*len(ads)

In [14]:
def find_indices(scores, threshold):
    return [i for i, score in enumerate(scores) if score > threshold]

In [15]:
def predict(requests, ads, threshold=0.5):
    predictions = {}
    for i, request in enumerate(requests):
        scores = calc_similarity(request, ads)
        indices = find_indices(scores, threshold)
        if indices:
            predictions[str(i+1)] = [str(idx+1) for idx in indices]  # indexation starts from 1 in the true_markup
    return predictions

In [16]:
pred_markup = predict(requests, ads, threshold=0.6)

# Evaluation

## Example Data

In [17]:
ex_true_markup = {"1": ["1", "4"],
     "3": ["7"],
     "4": ["3", "1", "5"]
    }

ex_pred_markup = {"1": ["1", "8", "4"],
     "3": ["7"],
     "5": ["10"]
    }

ex_n_ads = 10
ex_n_requests = 5

ex = """
1)  1 2 3 4 5 6 7 8 9 10
map 1 0 0 1 0 0 0 0 0 0 
pr  1 0 0 1 0 0 0 1 0 0

2)  1 2 3 4 5 6 7 8 9 10
map 0 0 0 0 0 0 0 0 0 0 
pr  0 0 0 0 0 0 0 0 0 0

3)  1 2 3 4 5 6 7 8 9 10
map 0 0 0 0 0 0 1 0 0 0 
pr  0 0 0 0 0 0 1 0 0 0

4)  1 2 3 4 5 6 7 8 9 10
map 1 0 1 0 1 0 0 0 0 0 
pr  0 0 0 0 0 0 0 0 0 0

5)  1 2 3 4 5 6 7 8 9 10
map 0 0 0 0 0 0 0 0 0 0 
pr  0 0 0 0 0 0 0 0 0 1

CONFUSION MATRIX:
TP: 3
FP: 2
TN: 42
FN: 3

METRICS
Acc:  0.9
Prec: 0.6
Rec:  0.5
F1:   0.(54)
"""

In [18]:
ex_confusion_matrix = metrics.calc_confusion_matrix(ex_true_markup, ex_pred_markup, n_ads=ex_n_ads, n_requests=ex_n_requests)
ex_confusion_matrix

{'TP': 3, 'FP': 2, 'TN': 42, 'FN': 3}

In [19]:
ex_stats = metrics.calc_all_stats(ex_confusion_matrix)
ex_stats

{'accuracy': 0.9, 'precision': 0.6, 'recall': 0.5, 'f1': 0.5454545454545454}

In [20]:
metrics.compare_with_saved_stats(ex_stats, ex_confusion_matrix)

-----------------------------------------------------------------------------------------
|	Metric		|	Old Value	|	New Value	|	Diff	|
-----------------------------------------------------------------------------------------
|	TP		|	55		|	3		|	📉 -52	|
|	FP		|	542		|	2		|	📉 -540	|
|	TN		|	86902		|	42		|	📉 -86860	|
|	FN		|	323		|	3		|	📉 -320	|
|	Prec		|	0.092		|	0.600		|	📈 0.508	|
|	Recall		|	0.146		|	0.500		|	📈 0.354	|
|	F1		|	0.113		|	0.545		|	📈 0.433	|

F1 📈 increased by 0.433, up to 54.5%, which is a significant growth 🚀


## Real Data

In [21]:
true_markup = dataset_utils.load_matching_data(matching_file)

In [22]:
confusion_matrix = metrics.calc_confusion_matrix(true_markup, pred_markup, n_ads=len(ads), n_requests=len(requests))
confusion_matrix

{'TP': 40, 'FP': 415, 'TN': 87028, 'FN': 339}

In [23]:
stats = metrics.calc_all_stats(confusion_matrix)
stats

{'accuracy': 0.9914144519596456,
 'precision': 0.08791208791208792,
 'recall': 0.10554089709762533,
 'f1': 0.0959232613908873}

In [24]:
metrics.compare_with_saved_stats(stats, confusion_matrix)

-----------------------------------------------------------------------------------------
|	Metric		|	Old Value	|	New Value	|	Diff	|
-----------------------------------------------------------------------------------------
|	TP		|	55		|	40		|	📉 -15	|
|	FP		|	542		|	415		|	📉 -127	|
|	TN		|	86902		|	87028		|	📈 126	|
|	FN		|	323		|	339		|	📈 16	|
|	Prec		|	0.092		|	0.088		|	📉 -0.004	|
|	Recall		|	0.146		|	0.106		|	📉 -0.040	|
|	F1		|	0.113		|	0.096		|	📉 -0.017	|

F1 📉 decreased by 0.017, down to 9.6%, which is within the margin of error.
